In [4]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('/data/github/mfmtk-utils/')
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from mfmtkutils import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Table of Contents
0. Initialization
1. Photometric Params
     * [Effective Radii Rn](#rnandn)
     * [The 'Spiking' Problem](#spiking)
     * [Effective Radii Part #2](#eraddi2)
     * [Sérsic Index n](#sersicindex)
2. Morphometric Params
    * [Concentration $C_1$](#c1)
    * [Concentration $C_2$](#c2)
3. Appendixes    
    * [Catalog Class](#appclass)
    * [Plotting Utilities](#appB)

# Initialization

In [5]:
path = '/data/catalogs/FERENGI/SDSS/flux/'


redshifts = ['0.02', '0.03', '0.04', '0.05', '0.06', '0.07', '0.08', '0.09', '0.10', '0.12', 
             '0.14', '0.16', '0.18', '0.20', '0.25', '0.30', '0.35', '0.40', '0.45']

T_type = np.loadtxt('/data/datasets/EFIGI_PGC.txt', usecols=[0, 1], dtype=str).T

Reduce and generate a vector in which each elements is a catalog for a given parameter

In [6]:
flux_cats = []
for z in redshifts:
    cat = catalog(path=path+z+'.mfmtk')
    flux_cats.append(cat)

Now flux_cats has information from each catalog

In [7]:
#Which parameters do we need? (in order)
photo_params = ['Mo', 'RnFit2D', 'nFit2D', 'Rp']
morpho_params = ['C1', 'C2', 'A1', 'A3', 'S1', 'S3',
                 'G', 'H', 'M20', 'sigma_psi']

Now we can reduce flux_cats to produce catalogs for each parameter, let's start with the photo ones

In [8]:
photo_catalogs = []
for j, param in enumerate(photo_params):
    flux = flux_cats[0].reduce(flux_cats, param).raw_catalog    
    photo_catalogs.append(flux[1:])
    
galaxies = flux_cats[0].raw_catalog[0]

And the same for the morpho ones

In [9]:
morpho_catalogs = []
for j, param in enumerate(morpho_params):
    flux = flux_cats[0].reduce(flux_cats, param).raw_catalog
    morpho_catalogs.append(flux[1:])

It takes a long time to run, but it's a single run for the entire time working under this NB. For example, let's work with the image size only to check our process

In [ ]:
z = np.array(redshifts).astype(float)

# Photometric Params
## Effective Radii $R_n$

<div id="rnandn"></div>
Now, let's work with the Effective Radii (RnFit2D)

In [ ]:
Mo = photo_catalogs[0].T.astype(float)
Rn = photo_catalogs[1].T.astype(float)

Finds which galaxy is spiral or elliptical

In [ ]:
E_indexes = T_type.T[np.where(T_type[1].astype(float) < -2)].T[0]
S_indexes = T_type.T[np.where((T_type[1].astype(float) >= 1) & (T_type[1].astype(float) < 7))].T[0]


lenticulars  = np.array([i for i, val in enumerate(galaxies) if val in set(S0_indexes)])
spirals = np.array([i for i, val in enumerate(galaxies) if val in set(S_indexes)])
ellipticals = np.array([i for i, val in enumerate(galaxies) if val in set(E_indexes)])
print '# Spirals:', spirals.shape[0]
print '# Ellipticals:', ellipticals.shape[0]

<div id="spiking"></div>
# Spiking Effect (Sérsic 2D fit errors)

What is the overall behavior of $R_n$?

In [ ]:
plt.plot(z, Rn.T, '-k', alpha=0.5);

This plot reveals some degree of error in our measurements. There is a spiking effect in some redshift slices, tracking those galaxies would be useful. We could do that by taking the derivative of our measurements and finding where there is a increase in the function since it is overall monotonic decreasing. Sure we would lose some information about ''natural'' increasing not due measurement erros, but in a whole this method works. We take the derivative for each galaxy, np.gradient would take the spatial derivative for all the 2D $R_n$ matrix. 

In [ ]:
spiked = []
not_spiked = []
for i, galaxy in enumerate(Rn):
    derivative = np.gradient(galaxy)
    if not (np.size(derivative[np.where(derivative > 0)]) > 0):
        not_spiked.append(i)
    else:
        spiked.append(i)
spiked = np.array(spiked)
not_spiked = np.array(not_spiked)

and just for the sake of simplicity

In [ ]:
spirals_not_spiked = intersect(spirals, not_spiked)
ellipticals_not_spiked = intersect(ellipticals, not_spiked)
print not_spiked.shape

And then we have

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))
ax1.plot(z, Rn[spiked].T, '-k')
ax2.plot(z, Rn[not_spiked].T, '-k');

Our objective is not to exclude these galaxies but to track them down and verify morfometryka measurement on them. galaxies[spiked] give us every one of them but we would want only worst cases for it is a lot more easier to find the problem with tem. We increased the threshold for the number of derivatives to 5, so only galaxies showing 5 spikes are allowed in our selection

We reduced our pool of galaxies from $\sim 500$ to $16$. Let's write them in a catalog for future use

In [ ]:
f = open('/data/worst_spiking_cases.mfmtk', 'w')
for gal in galaxies[spiked]:
    f.write(gal + '\n')
f.close()

<div id="eraddi2"></div>
# Effective Radii Part #2

We have to extract the resolution behavior by multiplying our Rn measurement by the scale factor $n$ that 
$$ S_{f} = S_{i} n $$
where S stands for 'Size'. The image is a square, width and height are the same. To find the right Rn we need to
$$ Rn = \frac{R_z}{n} $$
The initial image size is 1024, so
$$ n = \frac{Mo}{1024}  $$

In [ ]:
Rn_f = 1024 * Rn / Mo
f, axes = plt.subplots(4, 5, figsize=(15, 7))
plt.subplots_adjust(wspace=0, hspace=0)
histograms(Rn_f[spirals], z, axes)
histograms(Rn_f[ellipticals], z, axes, color='red')

The graph shows some spikes due to errors of measurement by Morfometryka (on Sérsic profile fits). The problem is that some galaxies becomes very faint in these redshift simulations and we're not able to performe realiable measurements, but working with a big sample diminish the effect produced by outliers. What if we make a linear regression for each galaxy and then see how the distribution of regressions behave? Let's do that for all galaxies first, then we could use the indexes in ''spirals'' and ''ellipticals'' to retrive them and treat them separately. So, we have something like this


In [ ]:
Rn_fits = []
for galaxy in Rn_f:
    fit = np.polyfit(z, galaxy, 1)
    Rn_fits.append(fit)

Rn_fits = np.array(Rn_fits)

Here it's trickier to plot all the fits in the same way we did before, let's plot them separately

In [ ]:
for fit in Rn_fits[spirals]:
    plt.plot(z, z*fit[0] + fit[1], '-b')
    
for fit in Rn_fits[ellipticals]:    
    plt.plot(z, z*fit[0] + fit[1], '-r')

Overwall, the spiking effect disappeared and the distribution in the bottom seems strong enough to give us some insights, let's see how the mean for each class behaves. Here, we use the ''axis'' argument from np.mean() to take the mean from each column of the data (each redshift step), the same is valid for the standard deviation. So each element of the new matrix is a tuple (mean, std) for each redshift step.

In [ ]:
RnS_stats = (Rn_f[spirals].T.mean(axis=1), Rn_f[spirals].T.std(axis=1))
RnE_stats = (Rn_f[ellipticals].T.mean(axis=1), Rn_f[ellipticals].T.std(axis=1))

We need statstics from our fits too. Let's plot only the classes distributions information, we'll use a little bit of jittering to distinguish the error bars (out standard deviation)

In [ ]:
plt.errorbar(z, RnS_stats[0], yerr=RnS_stats[1])
plt.errorbar(z + 0.005, RnE_stats[0], yerr=RnE_stats[1], color='red')

## Sérsic Index $n$

<div id="sersicindex"></div>
This validates to us that within some scattering, Morfometryka is able to retrieve the information about structural parameters (size, in this case) even in low SNR and resolution regimes. Now, we can apply this procedure to each of the parameters. Let's proceed to the Sérsic Index here called nFit2D.

In [ ]:
n = photo_catalogs[2].T.astype(float)
f, axes = plt.subplots(4, 5, figsize=(15, 7))
plt.subplots_adjust(wspace=0, hspace=0)
histograms(n[spirals_not_spiked], z, axes)
histograms(n[ellipticals_not_spiked], z, axes, color='red')

In [ ]:
n_fits = []
for galaxy in n:
    fit = np.polyfit(z, galaxy, 1)
    n_fits.append(fit)

n_fits = np.array(n_fits)

In [ ]:
for fit in n_fits[spirals]:
    plt.plot(z, z*fit[0] + fit[1], '-b')
    
for fit in n_fits[ellipticals]:    
    plt.plot(z, z*fit[0] + fit[1], '-r')

In [ ]:
nS_stats = (n[spirals].T.mean(axis=1), n[spirals].T.std(axis=1))
nE_stats = (n[ellipticals].T.mean(axis=1), n[ellipticals].T.std(axis=1))
plt.errorbar(z, nS_stats[0], yerr=nS_stats[1])
plt.errorbar(z + 0.005, nE_stats[0], yerr=nE_stats[1], color='red')
plt.xlim([0.02, 0.50])

# Morphometric Parameters

## Concentrations $C_1$ and $C_2$

$$ C_1 = \log_{10} \left ( \frac{R_{80}}{R_{20}} \right ) $$ $$ C_2 = \log_{10} \left ( \frac{R_{90}}{R_{50}} \right ) $$

In [ ]:
C1 = morpho_catalogs[0].astype(float).T/5
C2 = morpho_catalogs[1].astype(float).T/5

In [ ]:
%load_ext autoreload
%autoreload 2
from mfmtkutils import *
f, (ax1, ax2) = plt.subplots(1,2, sharey=True, sharex=True, figsize=(12, 5))
plt.subplots_adjust(wspace=0)
ax1.set_xlim([0.02, 0.45])
plot_as_gaussians(C1[spirals], z, ax1)
plot_as_gaussians(C1[ellipticals], z, ax1, color='red')
plot_as_gaussians(C2[spirals], z, ax2, label=r'$\rm Spirals$')
plot_as_gaussians(C2[ellipticals], z, ax2, color='red')

plt.legend(loc=1)

The $C_1$ distributions for each redshift are

In [ ]:
f, axes = plt.subplots(3, 7, figsize=(15, 5))
plt.subplots_adjust(wspace=0, hspace=0)
histograms(C1[spirals], z, axes, normed=1)
histograms(C1[ellipticals], z, axes, color='red', normed=1)

the same for $C_2$ gives

In [ ]:
f, axes = plt.subplots(3, 7, figsize=(15, 5))
plt.subplots_adjust(wspace=0, hspace=0)
histograms(C2[spirals], z, axes, normed=1)
histograms(C2[ellipticals], z, axes, color='red', normed=1)

Plotting bot classes for $C_1$ give us


Apparently it seems that both classes decreases linearly and the spiking is small here

In [ ]:
C1S_stats = (C1[spirals].T.mean(axis=1), C1[spirals].T.std(axis=1))
C1E_stats = (C1[ellipticals].T.mean(axis=1), C1[ellipticals].T.std(axis=1))
plt.errorbar(z, C1S_stats[0], yerr=C1S_stats[1])
plt.errorbar(z + 0.005, C1E_stats[0], yerr=C1E_stats[1], color='red')
plt.xlim([0.02, 0.25])

To make a more meaningful plot let's use every information we got. See [Plot Utilities](#appA) for more information about our plotting routines. Here we have two takes on density plots

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))

C1_fits = bulk_linear_fit(C1).T
plot_final(C1, C1S_stats, C1E_stats, C1_fits, spirals, ellipticals, ax=ax1)
density_mesh_plot(z, C1[ellipticals],cmap="Oranges", ax=ax2)
density_mesh_plot(z, C1[spirals], ax=ax2, mask_threshold=10)
ax1.set_xlim([0.02, 0.25])
ax2.set_xlim([0.02, 0.25])
ax1.set_ylim([2, 5])
ax2.set_ylim([2, 5])
ax2.set_yticks([])
plt.subplots_adjust(wspace=0)

As we can see, both linear fits describes the overall distribution very well. There's a lot of scattering, but filled region shows $1 \ \sigma$ 

The one in the left shows the means for both classes distribution of measurements with solid lines and $\pm  1 \sigma$ in the shaded area. Right plot show a box mesh density distribution. We would need to performe all steps for each of the parameters, how convinient would it be to plot everything at the same time and then follow with treatment only in the special cases? Here it goes. We want 10 plots, one for each parameter. Let's make 2 columns with 5 rows each

In [ ]:
f, axes = plt.subplots(5, 2, figsize=(10, 18))
labels = [r'$\rm C_1$', r'$\rm C_2$', r'$\rm A_1$', r'$\rm A_3$',
          r'$\rm S_1$', r'$\rm S_3$', r'$\rm G$', r'$\rm H$', r'$\rm M_20$', r'$\rm \sigma_\psi $']
morpho_params = ['C1', 'C2', 'A1', 'A3', 'S1', 'S3',
                 'G', 'H', 'M20', 'sigma_psi']

for i, ax in enumerate(axes.flat):
    param = morpho_catalogs[i].astype(float).T
    paramS_stats = (param[spirals].T.mean(axis=1), param[spirals].T.std(axis=1))
    paramE_stats = (param[ellipticals].T.mean(axis=1), param[ellipticals].T.std(axis=1))
    #param_fits = bulk_linear_fit(param).T
    plot_final(param, paramS_stats, paramE_stats, spirals, ellipticals, ax=ax)
    ax.set_xlim([0.02, 0.50])
    if(i < 8):
        ax.set_xticks([])
    else:
        ax.set_xlabel(r'$\rm Redshift$', fontsize=20)
    ax.set_ylabel(labels[i], fontsize=20)
    if not (i % 2 == 0):
        ax.yaxis.tick_right()
        ax.yaxis.set_label_position("right")
    
plt.subplots_adjust(hspace=0, wspace=0)    

So, as we can see, only the last three plots got problems, we now know about that and can proceed to work in each at the time

We could make a linear fit for each class data and use it's standard deviation as follows

<div id="appclass"> </div>
# Appendix A: The Catalog class

<div id="appB"></div>
# Appendix B: Plotting Utilities

In [ ]:
def plot_final(C1, C1S_stats, C1E_stats, spirals, ellipticals, ax=False, points=False):
    if not ax:
        if(points):
            #Plot Points with Jittering
            for gal in C1[spirals]:
                plt.plot(z+0.1*z*np.random.rand(), gal, 'xb', alpha=0.2)

            for gal in C1[ellipticals]:
                plt.plot(z+0.1*z*np.random.rand(), gal, 'vr', alpha=0.2)

        #Plot means and fits
        plt.plot(z, C1S_stats[0], '--k', lw=2)
        plt.plot(z, C1E_stats[0], '--k')
        


        plt.fill_between(z, C1S_stats[0] - C1S_stats[1],
                         C1S_stats[0] + C1S_stats[1], alpha=0.5)
        
        plt.fill_between(z, C1E_stats[0] - C1E_stats[1],
                         C1E_stats[0] + C1E_stats[1], alpha=0.5, facecolor='red')
        
    else:
        if(points):
            for gal in C1[spirals]:
                ax.plot(z+z*np.random.rand(), gal, 'xb', alpha=0.2)

            for gal in C1[ellipticals]:
                ax.plot(z+z*np.random.rand(), gal, 'vr', alpha=0.2)

        #Plot means and fits
        ax.plot(z, C1S_stats[0], '--k', lw=2)
        ax.plot(z, C1E_stats[0], '--k')

        
        ax.fill_between(z, C1S_stats[0] - C1S_stats[1],
                         C1S_stats[0] + C1S_stats[1], alpha=0.5)
        
        ax.fill_between(z, C1E_stats[0] - C1E_stats[1],
                         C1E_stats[0] + C1E_stats[1], alpha=0.5, facecolor='red')
        
def bulk_linear_fit(param):
    param_fits = []
    for galaxy in param:
        fit = np.polyfit(z, galaxy, 1)
        param_fits.append(fit)
    
    return np.array(param_fits)

def plot_linear_fit(galaxies, params='-b'):
    for fit in galaxies:
        plt.plot(z, z*fit[0] + fit[1], params)
    
    
def parse_to_points(x, y):
    xs = []
    ys = []
    for c1 in y:
        for zi, ci in zip(z, c1):
            xs.append(zi + 0.5*zi*np.random.rand())
            ys.append(ci)
    
    return xs, ys


def density_mesh_plot(x, y, cmap="Blues", mask_threshold=1, ax=None):
    y[np.where(np.isnan(y))] = 0
    x_p, y_p = parse_to_points(x, y)
    H, xedges, yedges = np.histogram2d(x_p, y_p,  bins=(60, 20))
    H = np.rot90(H)
    H = np.flipud(H)
    H = np.ma.masked_array(H, H < mask_threshold)
    if(ax):
        ax.pcolormesh(xedges, yedges, H, cmap=cmap)
    else:
        plt.pcolormesh(xedges, yedges, H, cmap=cmap)
    